In [1]:
#default_exp metrics

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#hide
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

# Dev comments

- [X] TODO: make entropy estimator with KDE
- [X] TODO: make quantile func
- [ ] TODO: make Gaussian Mixture entropy estimation

# Imports -

In [7]:
#export
import numpy as np
from sklearn.metrics import pairwise
from scipy import stats
from sklearn.preprocessing import QuantileTransformer

from scikit_density.core.random_variable import KDE, RandomVariable
from scikit_density.utils import (
    _fix_one_sample_2d, _fix_one_dist_2d, _fix_dist_1d,
    _fix_X_1d, _assert_dim_3d, _assert_dim_1d, _assert_dim_2d, _fix_one_dist_1d
)


array([[ 8.50929219e-01, -1.28019761e-01],
       [ 2.92724507e-01, -8.50980853e-02],
       [-6.94117028e-01, -1.12725161e+00],
       [-3.08428075e-01, -6.04288132e-01],
       [ 4.65298655e-01,  5.75556350e-01],
       [-6.55849763e-01,  1.83692045e+00],
       [ 2.28571355e-01, -4.48161387e-01],
       [-7.86980916e-01, -6.08785650e-01],
       [-7.94334083e-02, -8.80467503e-01],
       [ 1.14693587e+00, -6.01776398e-01],
       [-1.61151685e+00,  2.67085474e-02],
       [ 1.58999219e+00, -8.08981717e-01],
       [-1.60985591e-01, -9.84344616e-01],
       [ 1.76712606e+00,  1.05808375e+00],
       [-4.47777710e-01, -9.10271476e-02],
       [ 5.06028462e-01, -8.61489673e-01],
       [ 8.38234211e-01, -9.15954035e-01],
       [ 9.07776461e-01,  1.19285987e+00],
       [ 1.25942921e-01, -1.83435660e+00],
       [-1.53645458e+00, -2.19237688e-01],
       [-1.16829680e+00, -1.39074050e+00],
       [-2.15197116e-02,  4.85128085e-01],
       [ 1.55637164e-01, -2.24331626e+00],
       [-3.

# Test data

We are going to test utils using the following data:

In [642]:
#documentation imports
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [13]:
#data
y_dists = np.random.randn(10,200,2)
y_true = np.random.randn(10,1,2)

y_dists_1d = np.random.randn(10,200,1)
y_true_1d = np.random.randn(10,1,1)

# `Metrics` -

## `kde_entropy`

In [14]:
#export
def kde_entropy(data, sample_size = 10000, **kde_kwargs):
    '''
    Calculates the entropy of multiple continuous distributions. entropy equals np.mean(-np.log(p(x)))
    input should be of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)
    '''
    data = _assert_dim_3d(data)
    return np.array([KDE(**kde_kwargs).fit(d).entropy(sample_size = sample_size) for d in data])

`kde_entropy` estimates a distribution of an aribtrary dimension random variable using kernel density estimation and calculates its entropy

In [15]:
kde_entropy(y_dists)

array([4.31820291, 4.29283959, 4.35741535, 4.34262252, 4.19655634,
       4.3395498 , 4.14877126, 4.33761258, 4.40963779, 4.21663598])

In [16]:
%%timeit
kde_entropy(y_dists)

300 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## `kde_likelihood`

In [17]:
#export
def kde_likelihood(y_true,samples, **kde_kwargs):
    '''
    Calculates the likelihood of y_true in kde estimation of samples
    input should be of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)
    '''
    data = _assert_dim_3d(samples)
    return np.array([KDE(**kde_kwargs).fit(samples[i]).evaluate(y_true[i:i+1]) for i in range(samples.shape[0])])

`kde_likelihood` Calculates the likelihood of y_true in kde estimation of samples

In [18]:
kde_likelihood(y_true,y_dists)

array([[0.11703671],
       [0.11222167],
       [0.01766762],
       [0.02654011],
       [0.18783367],
       [0.15143951],
       [0.0343707 ],
       [0.02111608],
       [0.03682947],
       [0.19200231]])

In [19]:
%%timeit
kde_likelihood(y_true,y_dists)

7.37 ms ± 97.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## relative_likelihood

In [ ]:
def relative_likelihood

## `naive_bimodal_gaussian_likelihood`

In [ ]:
#ESTIMATE A BIMODAL GAUSSIAN BASED ON MEAN AND STD OF BLOBS AFTER BIMODAL_SPLITR

## `quantile`

In [647]:
#export
def quantile(y_true, pred_dist):
    '''checks in which quantile lies y_true, given the predicted distribution'''    
    y_true = _fix_X_1d(y_true)
    y_true = _fix_one_sample_2d(y_true)
    pred_dist = _assert_dim_3d(pred_dist)
    assert y_true.shape[0] == pred_dist.shape[0], 'number of dists should be the same as number of points'
    return _fix_one_dist_2d(np.array([(y_true[i].T >= pred_dist[i].T).mean(axis = 1) for i in range(len(y_true))]))

`quantile` checks the quantile of each dimension of an observation `y_true` given an empirical distribution `y_dists`

In [648]:
quantile(y_true, y_dists)

array([[[0.725, 0.015],
        [0.945, 0.695],
        [0.54 , 0.17 ],
        [0.05 , 0.34 ],
        [0.77 , 0.11 ],
        [0.625, 0.705],
        [0.925, 0.24 ],
        [0.705, 0.805],
        [0.99 , 0.67 ],
        [0.18 , 0.655]]])

In [649]:
%%timeit
quantile(y_true, y_dists)

150 µs ± 3.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## `quantile_sklearn`

In [650]:
#export
def quantile_sklearn(y_true, pred_dist):
    '''checks in which quantile lies y_true, given the predicted distribution, using skleaerns QuantileTransformer'''
    y_true = _fix_X_1d(y_true)
    y_true = _fix_one_sample_2d(y_true)
    assert y_true.shape[0] == pred_dist.shape[0], 'number of dists should be the same as number of points'
    return np.array([QuantileTransformer().fit(pred_dist[i]).transform(y_true[i]) for i in range(pred_dist.shape[0])])

`quantile_sklearn` does the same as `quantile`, but with `sklearn`s `QuantileTransformer`

In [651]:
quantile_sklearn(y_true, y_dists)

array([[[0.7278008 , 0.01447408]],

       [[0.9496229 , 0.69639994]],

       [[0.53860615, 0.1690896 ]],

       [[0.04560922, 0.34077751]],

       [[0.76982081, 0.10779354]],

       [[0.62630501, 0.70365157]],

       [[0.92464003, 0.24046056]],

       [[0.70444373, 0.80482178]],

       [[0.99463003, 0.66871387]],

       [[0.17602332, 0.65460287]]])

In [652]:
%%timeit
quantile_sklearn(y_true, y_dists)

5.43 ms ± 85.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## `theoretical_entropy`

In [660]:
#export
def theoretical_entropy(data, dist = 'norm'):
    '''
    returns the entropy of the maximum likelihood estiation of `dist` given `data`
    dist should be one dimensional for cases when `dist` != 'kde'
    '''    
    data = _assert_dim_3d(data)    
    return np.array([RandomVariable(d).fit_dist(dist).entropy(dist) for d in data])

`theoretical_entropy` returns the entropy of the maximum likelihood estiation of `dist` given `data`
    dist should be one dimensional for cases when `dist` != 'kde'

In [668]:
theoretical_entropy(y_dists_1d, dist = 'norm')

array([1.30701586, 1.51849768, 1.49686731, 1.49346887, 1.33472044,
       1.33898642, 1.3316598 , 1.44444205, 1.43230334, 1.49410763])

In [666]:
%%timeit
theoretical_entropy(y_dists_1d, dist = 'norm')

6.9 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## `marginal_variance`

In [639]:
#export
def marginal_variance(data):
    '''
    Calculates the variance for each dimension (marginal) of multiple distributions
    input should be of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)
    '''
    data = _assert_dim_3d(data)
    return data.var(axis = -2)

`marginal_variance` Calculates the variance for each dimension (marginal) of multiple distributions
    input should be of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)

In [640]:
marginal_variance(y_dists)

array([[1.07997805, 0.97298247],
       [1.08478207, 0.80053107],
       [1.04474869, 0.90155785],
       [1.06384356, 0.96478837],
       [1.05452459, 0.91047929],
       [0.90663148, 1.10136982],
       [1.01474701, 0.95389312],
       [0.86083731, 0.94958826],
       [0.96977624, 1.11934001],
       [0.92623992, 1.04994311]])

## `covariance_matrix` 

In [ ]:
#export
def covariance_matrix(data):
    '''
    Calculates the variance for each dimension (marginal) of multiple distributions
    input should be of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)
    '''
    raise NotImplementedError

## `bimodal_split`

In [672]:
#export    
def bimodal_split(data, filter_size = 3, lb = 0.1,ub = 0.9):
    '''
    reutrns siplitting point of single distribution in two according to the highest value of the derivative of cpdf
    '''
    
    _assert_dim_1d(data)
    filter_size = int(np.ceil(filter_size)) if filter_size >= 1 else int(max(1,np.ceil(data.shape[0]*filter_size)))
    data.sort()
    data = filter_borders(data,lb,ub)
    diff = np.diff(data)    
    v = np.ones(filter_size)/filter_size    
    diff = np.convolve(a = diff,v = v, mode = 'same')
    argmax = np.argmax(diff)
    split_point = (data[min(data.shape[0] - 1,argmax + filter_size)] + data[argmax])/2
    return split_point

`bimodal_split` reutrns siplitting point of single distribution in two according to the highest value of the derivative of cpdf

In [675]:
bimodal_split(y_dists[0,:,0], filter_size = 3, lb = 0.1,ub = 0.9)

-1.4383953140490906

In [678]:
%%timeit
bimodal_split(y_dists[0,:,0], filter_size = 3, lb = 0.1,ub = 0.9)

22.4 µs ± 667 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## `mad` 

In [676]:
#export
def mad(data):
    '''
    calculates mean average deviance (a robust version of standard deviation)
    '''
    np.median(np.absolute(data - np.mean(data)))
    pass

## `bimodal_variance` 

In [8]:
#export
def bimodal_variance(data, filter_size = 0.05, lb = 0.1,ub = 0.9):
    '''
    returns weighted marginal variance of splitted data in two according to the highest value of the derivative of cpdf
    returns a variance array of shape (n_dists, n_dims)
    '''
    data = _assert_dim_3d(data)    
    #make split point
    #GENERALIZE FOR MULTIDIM
    mses = []
    for dist in range(data.shape[0]):        
        mses.append([])
        for dim in range(data.shape[2]):            
            d = data[dist,:,dim]
            split_point = bimodal_split(d, filter_size, lb ,ub)
            arr1, arr2 = d[d >= split_point], d[d < split_point]
            mses[dist].append((arr1.shape[0]*(arr1.var()) + arr2.shape[0]*(arr2.var()))/d.shape[0])                            
    #average variance    
    return np.array(mses)

`bimodal_variance` returns weighted variance of splitted data in two according to the highest value of the derivative of cpdf
    returns a variance array of shape (n_dists, n_dims)

In [9]:
bimodal_variance(y_dists, filter_size = 0.05, lb = 0.1,ub = 0.9)

NameError: name 'y_dists' is not defined

In [684]:
%%timeit
bimodal_var(y_dists, filter_size = 0.05, lb = 0.1,ub = 0.9)

1.31 ms ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## `gaussian_distance_entropy`

In [ ]:
#export
def gaussian_distance_entropy(data):
    '''
    calculates the entropy of the distribution of distances from centroid of points in dist, assuming normal distribution
    '''
    return -np.log(expected_distance_gaussian_likelihood(data))

## `expected_distance_gaussian_likelihood`

In [ ]:
#export
def expected_distance_gaussian_likelihood(data):
    '''
    calculates the expected likelihood of the distances from centroid of samples in distributions in dist
    input should be of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)
    '''
    dist = _assert_dim_3d(data)    
    return np.array([distance_gaussian_likelihood(d).mean() for d in data])

## `distance_gaussian_likelihood`

In [600]:
#export
def distance_gaussian_likelihood(data):    
    '''
    calculates the expected likelihood of the distances from the centroid of samples in dist
    '''
    centroid = data.mean(axis = 0).reshape(1,-1)
    distances =  pairwise.euclidean_distances(data, centroid).flatten()  
    distance_std = distances.std()
    if distance_std == 0:
        return 1
    z = (distances - distances.mean())/distance_std
    return 1/(distance_std*np.pi**(1/2))*np.exp(-1/2*z**2)

# Helping functions - 

## `make_outlier_filter`

In [687]:
#export
def make_outlier_filter(data, lb = 25, ub = 75, c = 1):
    a = np.array(data)
    upper_quartile = np.percentile(a, ub)
    lower_quartile = np.percentile(a, lb)
    iqr = (upper_quartile - lower_quartile) * c
    lb = lower_quartile - iqr
    ub = upper_quartile + iqr
    filter_ = np.zeros(a.shape)
    filter_[(a >= lb) & (a <= ub)] = 1
    return filter_

## `filter_borders`

In [53]:
#export
def filter_borders(data, lb = 0.05, ub = 0.95):
    lb = int(len(data)*lb)
    ub = int(len(data)*ub)
    return data[lb:ub]

#def distance_log_variance(dist):
#    '''variance of the distances of points to centroid of distribution'''
#    centroid = dist.mean(axis = 0).reshape(1,-1)
#    distances =  pairwise.euclidean_distances(dist, centroid).flatten()    
#    return distances.var()

# Export -

In [3]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_ensemble.ipynb.
Converted 02_core.random_variable.ipynb.
Converted 03_utils.ipynb.
Converted 04_metrics.ipynb.
Converted 05_neighbors.ipynb.
Converted 06_kde_baesyan_nets.ipynb.
Converted index.ipynb.


In [22]:
#hide
from nbdev.showdoc import show_doc
show_doc(kde_entropy)

<h4 id="kde_entropy" class="doc_header"><code>kde_entropy</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>kde_entropy</code>(**`data`**, **`sample_size`**=*`10000`*, **`bw`**=*`'ISJ'`*)

Calculates the entropy of multiple distributions
input should be of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)